# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку

## Обзор данных

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

In [5]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`

In [9]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

 В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.

In [11]:
data['days_employed'] = data['days_employed'].abs()

In [12]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [14]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [15]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [18]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median() # заполняем пропуски медианными значениями

In [19]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [20]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [21]:
data['education'] = data['education'].str.lower()

In [22]:
data.duplicated().sum()

71

In [23]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [25]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [26]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаём функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [28]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [29]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование гипотез

In [30]:
data_grouped = data.groupby('children').agg({'debt': ['count', 'sum']})
data_grouped['conversion'] = (data_grouped['debt']['sum'] / data_grouped['debt']['count'])*100
print(data_grouped.sort_values(by='conversion', ascending=True))

           debt       conversion
          count   sum           
children                        
5             9     0   0.000000
0         14091  1063   7.543822
3           330    27   8.181818
1          4808   444   9.234609
2          2052   194   9.454191
4            41     4   9.756098


**Вывод:**  Должники с 4 и более детей мало информации, что бы сделать однозначные выводы. Все остальные данные показывают, что более надежные клиенты без детей, хоть и не сильно. 
 Дети накладывают дополнительные непрогнозируемые расходы. Хотя, нельзя утверждать однозначно, что количество детей определяет дисциплинированность, так как клиенты с тремя детьми белее надежные чем с двумя и одним (это может быть погрешность). Следовательно, более надежные клиенты, без детей.

#### Зависимость между семейным положением и возвратом кредита в срок?

In [32]:
data_grouped = data.groupby('family_status').agg({'debt': ['count', 'sum']})
data_grouped['conversion'] = (data_grouped['debt']['sum'] / data_grouped['debt']['count'])*100
print(data_grouped.sort_values(by='conversion', ascending=True))

                        debt      conversion
                       count  sum           
family_status                               
вдовец / вдова           951   63   6.624606
в разводе               1189   84   7.064760
женат / замужем        12261  927   7.560558
гражданский брак        4134  385   9.313014
Не женат / не замужем   2796  273   9.763948


**Вывод:**  В категории семейное положение все очень предсказуемо, наиболее дисциплинированные клиенты вдовцы / вдовы. здесь прослеживается возрастная корреляция. Работающие, средне старшего возраста, более ответственны. А категория не женаты / не замужем эта возрастная категория 18+. 

#### Зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
data_grouped = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})
data_grouped['conversion'] = (data_grouped['debt']['sum'] / data_grouped['debt']['count'])*100
print(data_grouped.sort_values(by='conversion', ascending=True))

                        debt       conversion
                       count   sum           
total_income_category                        
D                        349    21   6.017192
B                       5014   354   7.060231
A                         25     2   8.000000
C                      15921  1353   8.498210
E                         22     2   9.090909


**Вывод:** Людей с уровнем дохода А и Е учитывать нет смысла, мало данных. Дисциплинированная категория D с доходом 30 000 до 50 000. Категория с самым большим значением задолженостей по оплате это "С" с даходом от 50 000 до 200 000.

####  Влияние целей на возврат кредита в срок?

In [35]:
data_grouped = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})
data_grouped['conversion'] = (data_grouped['debt']['sum'] / data_grouped['debt']['count'])*100
print(data_grouped.sort_values(by='conversion', ascending=True))

                           debt      conversion
                          count  sum           
purpose_category                               
операции с недвижимостью  10751  780   7.255139
проведение свадьбы         2313  183   7.911803
получение образования      3988  369   9.252758
операции с автомобилем     4279  400   9.347978


**Вывод:** Операции с недвижимостью довольно большой процент от всех кредитов, самые дисциплинированные эта операции с недвижимостью.

#### Возможные причины появления пропусков в исходных данных


*Ответ:* Изходя из аналица таблице причина пропуска стал человечиский фактор, например поле dob_years принемает значение '0'. Так же давольно подозрительно выглядет то что колличесво недастоющих данных в двух столбцаз ровно по 2174. Возможно это говорит о технологическом сбое работы БД при выгрузке, или же как вариант сбой в работе программы во  время внесения данных.

#### Объяснение, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:*  есть три пути действия:
на право: ни чего не делать с данными оставит как есть, но тогда будут мешать пустые ячейки в таблице. Например при подсчете средних значений пустые ячейки не участвуютб таким образом среднее значение будет сильно отличаться.
Прямо: Удалить строки с пустыми ячейками. Можно если их не очень много и нет способа их заполнить.
На лево: Заменить значения на заранее подготовленные значения. Тут два варианта или среднее или медианное, так как у нас большой "разлет" по граничным значениям, то средние значения не подходят, в этом случае больше подходит медианные.

### Общий вывод.

В данной работе были проанализированы данные от банка - сведения о платежеспособности клиентов - с целью выявления зависимости выплаты кредита в срок от уровня дохода, количества детей, семейного положения и цели кредита.  
  
1) В результате проведенного исследования не удалось оценить данные групп клиентов с 4 и 5 детьми - выборка оказалась слишком малой для анализа.  
Самая многочисленная и надежная категория клиентов "без детей" оказалось самая надежная с самым маленьким процентов по задержкам оплаты - 7.54%. Самые не надежные с самым высоким процентом по задержкам оказались клиенты категории два ребёнка и составляет 9.45%.  

2) В результате анализа получены следующие результаты, самые добросовестные клиенты относятся к категории "вдовец / вдова" с процентом в задержек выплат 6.62%. Категория с самым высоким процентом задержек выплат "Не женат / не замужем" с процентом задержек выплат 9.76%.  

3) По результатам проверки выявлена следующая закономерность категория с самым маленьким процентом задержек "В" это категория с доходом от 30 000 до 50 000 значение 6%. Категория с малым большим процентом задержки с доходом от 50 000 до 200 000 значение 8.49%. Категория А с доходом от 1 000 000 и категория Е  от 0  до 30 000 не релевантны, слишком мало данных.

4) По результатам проверки предположения прослеживается следующая закономерность - меньше всего просрочек по кредитам по займам на недвижимость 7.26%, больше всего по займам на автомобиль 9.35%. Операции с недвижимостью довольно большой процент от всех кредитов.  
Выборка крайне не сбалансирована. Рекомендуется провести дополнительные исследования.ования.